# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [36]:
import pandas as pd

In [37]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [38]:
data.info()
data.describe()
#нет пустых значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.8+ MB


,user
count,289955.000000
mean,9852.460447
std,5692.355041
min,1.000000
25%,4935.000000
50%,9838.000000
75%,14769.000000
max,19718.000000


In [39]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']

In [40]:
#оставили в датасете нужные страны
data = data.loc[data['country'].isin(countries)]
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [41]:
#удалим строки с артистом '[unknown]'
data = data.drop(data[data['artist']=='[unknown]'].index)
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [42]:
data_by_invoice = data.groupby(['user', 'country'])['artist'].apply(lambda x: ','.join(x)).reset_index()
data_by_invoice

,user,country,artist
0,6,Portugal,"lily allen,kanye west,sigur rós,pink floyd,ste..."
1,12,Italy,"queen,the beatles,lynyrd skynyrd,neil young,th..."
2,36,Italy,"m.i.a.,fugazi,johnny cash,clint mansell,elton ..."
3,47,Italy,"pj harvey,radiohead,zero 7,einstürzende neubau..."
4,74,Spain,"in extremo,nine inch nails,billy talent,panic!..."
...,...,...,...
1412,19690,Italy,"pink floyd,the beatles,the white stripes,depec..."
1413,19693,Belgium,"air,simon & garfunkel,pixies,serge gainsbourg,..."
1414,19697,Belgium,"underworld,pendulum,the prodigy,sufjan stevens..."
1415,19704,Italy,"keane,pinback,coldplay,travis,calexico,doves,d..."


In [43]:
data = pd.concat([data_by_invoice.drop(['country','artist'], axis=1), 
                             pd.get_dummies(data_by_invoice['country']),
                             data_by_invoice['artist'].str.get_dummies(',')
                            ], axis = 1).set_index(['user'])
data

,Belgium,France,Italy,Portugal,Spain,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Работа с правилами

In [44]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets_apriori = apriori(data, min_support=0.03, use_colnames=True)
frequent_itemsets_apriori

,support,itemsets
0,0.112915,(Belgium)
1,0.230769,(France)
2,0.214538,(Italy)
3,0.086097,(Portugal)
4,0.355681,(Spain)
...,...,...
307,0.035992,"(the beatles, the rolling stones)"
308,0.031757,"(the beatles, the strokes)"
309,0.039520,"(the killers, the strokes)"
310,0.031757,"(coldplay, the killers, Spain)"


In [45]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets_fpgrowth = fpgrowth(data, min_support=0.05, use_colnames=True)
frequent_itemsets_fpgrowth.head()

,support,itemsets
0,0.174312,(muse)
1,0.130558,(pink floyd)
2,0.123500,(metallica)
3,0.096683,(depeche mode)
4,0.088215,(u2)


In [46]:
from mlxtend.frequent_patterns import fpmax
frequent_itemsets_fpmax = fpmax(data, min_support=0.04, use_colnames=True)
frequent_itemsets_fpmax

,support,itemsets
0,0.040226,(rihanna)
1,0.040226,(deus)
2,0.040226,(miles davis)
3,0.040932,(koЯn)
4,0.042343,(bob marley & the wailers)
...,...,...
125,0.071277,"(coldplay, Spain)"
126,0.083275,"(coldplay, radiohead)"
127,0.060692,"(Italy, radiohead)"
128,0.062103,"(France, radiohead)"


In [47]:
rules_lift = association_rules(frequent_itemsets_apriori, metric="lift", min_threshold=0.5).sort_values(by='lift', ascending=False)
rules_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
168,(kaiser chiefs),(franz ferdinand),0.050812,0.095977,0.030346,0.597222,6.222529,0.025469,2.244470
169,(franz ferdinand),(kaiser chiefs),0.095977,0.050812,0.030346,0.316176,6.222529,0.025469,1.388060
188,(kaiser chiefs),(the killers),0.050812,0.110092,0.031052,0.611111,5.550926,0.025458,2.288336
189,(the killers),(kaiser chiefs),0.110092,0.050812,0.031052,0.282051,5.550926,0.025458,1.322084
105,(bloc party),(interpol),0.083275,0.079746,0.032463,0.389831,4.888406,0.025822,1.508194
...,...,...,...,...,...,...,...,...,...
52,(nirvana),(Spain),0.124912,0.355681,0.037403,0.299435,0.841864,-0.007026,0.919713
48,(metallica),(Spain),0.123500,0.355681,0.035286,0.285714,0.803288,-0.008641,0.902047
49,(Spain),(metallica),0.355681,0.123500,0.035286,0.099206,0.803288,-0.008641,0.973030
61,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546


In [48]:
#rules_lift.head(5)
#любителям Franz Ferdinand и  The killer можно посоветовать Kaiser chiefs (т.к. KC менее популярны, чем FF и TK)
rules_lift[(rules_lift['lift'] > 5.0) & ( rules_lift['conviction'] < 2.0)].head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
169,(franz ferdinand),(kaiser chiefs),0.095977,0.050812,0.030346,0.316176,6.222529,0.025469,1.388060
189,(the killers),(kaiser chiefs),0.110092,0.050812,0.031052,0.282051,5.550926,0.025458,1.322084


In [49]:
rules_lift2 = rules_lift[(rules_lift['consequents'] != frozenset('Spain'))
                         &(rules_lift['consequents'] != frozenset('France'))
                         &(rules_lift['consequents'] != frozenset('Portugal'))
                         &(rules_lift['consequents'] != frozenset('Belgium'))
                         &(rules_lift['consequents'] != frozenset('Italy'))
                        & (rules_lift['support'] < 0.1)
                        ].sort_values(by='lift', ascending=True)
rules_lift2.head(1)
#Radiohead не стоит ехать с большими концертами в Испанию, они там не особо популярны

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
60,(Spain),(radiohead),0.355681,0.237826,0.062809,0.176587,0.742505,-0.021782,0.925628


In [50]:
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, min_threshold=0.02, metric="confidence")
rules_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
1,(Spain),(muse),0.355681,0.174312,0.067749,0.190476,1.092732,0.005749,1.019968
2,(muse),(radiohead),0.174312,0.237826,0.080452,0.461538,1.940653,0.038996,1.415465
3,(radiohead),(muse),0.237826,0.174312,0.080452,0.338279,1.940653,0.038996,1.247789
4,(muse),(coldplay),0.174312,0.188426,0.068454,0.392713,2.084171,0.035610,1.336391
5,(coldplay),(muse),0.188426,0.174312,0.068454,0.363296,2.084171,0.035610,1.296816
6,(pink floyd),(radiohead),0.130558,0.237826,0.055046,0.421622,1.772813,0.023996,1.317777
7,(radiohead),(pink floyd),0.237826,0.130558,0.055046,0.231454,1.772813,0.023996,1.131282
8,(Italy),(radiohead),0.214538,0.237826,0.060692,0.282895,1.189501,0.009669,1.062848
9,(radiohead),(Italy),0.237826,0.214538,0.060692,0.255193,1.189501,0.009669,1.054585


In [57]:
radiohead_lovers = rules_fpgrowth[
        (rules_fpgrowth['antecedents'] == {'radiohead'})
        & ((rules_fpgrowth['consequents'] == {'Spain'})
            | (rules_fpgrowth['consequents'] == {'Portugal'})
            | (rules_fpgrowth['consequents'] == {'France'})
            | (rules_fpgrowth['consequents'] == {'Italy'})
            | (rules_fpgrowth['consequents'] == {'Belgium'}))
    ].sort_values(by='lift', ascending=False)
radiohead_lovers

#наиболее популярны radiohead в Италии и Франции

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(radiohead),(Italy),0.237826,0.214538,0.060692,0.255193,1.189501,0.009669,1.054585
17,(radiohead),(France),0.237826,0.230769,0.062103,0.261128,1.131553,0.007220,1.041087
15,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546


In [66]:
exceptions = rules_fpgrowth[rules_fpgrowth['conviction'] < 1.0]
exceptions
#т.к. conviction меньше 1, можно сказать, что данные правила скорее исключения (такие правила не работают)
#lift < 1 => элементы чаще встречаются по отдельности
# => the beatles и radiohead в Испании не популярны

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(the beatles),(Spain),0.174312,0.355681,0.055752,0.319838,0.899227,-0.006248,0.947302
13,(Spain),(the beatles),0.355681,0.174312,0.055752,0.156746,0.899227,-0.006248,0.979169
14,(Spain),(radiohead),0.355681,0.237826,0.062809,0.176587,0.742505,-0.021782,0.925628
15,(radiohead),(Spain),0.237826,0.355681,0.062809,0.264095,0.742505,-0.021782,0.875546


In [76]:
for_nirvana_lovers = rules_lift[(rules_lift['antecedents'] == {'nirvana'}) & (rules_lift['lift'] > 2.0)]
for_nirvana_lovers
# любителям nirvana понравились бы red hot chili peppers и metallica

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
235,(nirvana),(red hot chili peppers),0.124912,0.10374,0.033169,0.265537,2.559630,0.020210,1.220292
202,(nirvana),(metallica),0.124912,0.12350,0.034580,0.276836,2.241582,0.019153,1.212035
